In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pynrrd nibabel


In [3]:
import os
import glob
import nrrd #pip install pynrrd, if pynrrd is not already installed
import nibabel as nib #pip install nibabel, if nibabel is not already installed
import numpy as np

# Specify the directory containing .nrrd files
input_dir = '/content/drive/MyDrive/Neuro/CODE/out'
# Check if the directory exists and list its contents
if os.path.exists(input_dir):
    print("Directory contents:", os.listdir(input_dir))
else:
    print("Directory does not exist. Check the path.")

# Get a list of .nrrd files in the directory
nrrd_files = glob.glob(os.path.join(input_dir, '*.nrrd'))
# Print the list of nrrd files found
print("NRRD files found:", nrrd_files)

Directory contents: ['1978753.nrrd']
NRRD files found: ['/content/drive/MyDrive/Neuro/CODE/out/1978753.nrrd']


In [4]:
# Prepare to log files with non-positive zooms
skipped_files = []

# Loop through each .nrrd file, only if the list is not empty
if nrrd_files:
    for nrrd_file in nrrd_files:
        try:
            # Read the .nrrd file
            data, header = nrrd.read(nrrd_file)

            # Extract zoom values and check if they are positive
            zooms = [
                header['space directions'][0, 0],
                header['space directions'][1, 1],
                header['space directions'][2, 2]
            ]

            if any(z <= 0 for z in zooms):
                # Log this file for further review
                skipped_files.append(nrrd_file)
                continue  # Skip the current file

            # Create a NIfTI1Image object
            nifti_img = nib.Nifti1Image(data, affine=None)

            # Update the NIfTI header with necessary information
            nifti_img.header.set_data_dtype(data.dtype)
            nifti_img.header.set_zooms(zooms)

            # Generate the output .nii file path by replacing the extension
            nii_file = nrrd_file.replace('.nrrd', '.nii.gz')

            # Save the NIfTI1Image object as .nii file
            nib.save(nifti_img, nii_file)
        except Exception as e:
            print(f"Error processing {nrrd_file}: {str(e)}")
else:
    print("No NRRD files found. Please check the directory and file extensions.")

# Report the skipped files
if skipped_files:
    print("Skipped files with non-positive zooms:")
    for file in skipped_files:
        print(file)

Skipped files with non-positive zooms:
/content/drive/MyDrive/Neuro/CODE/out/1978753.nrrd


In [ ]:
import nibabel as nib
import numpy as np

# Path to your NIfTI segmentation file
file_path = '/content/drive/MyDrive/Neuro/CODE/seg/1075451.nii.gz'

# Load the NIfTI file
nifti_img = nib.load(file_path)

# Get the data array from the NIfTI file
data_array = nifti_img.get_fdata()

# Find unique labels in the segmentation
unique_labels, counts = np.unique(data_array, return_counts=True)

# Display the labels and their counts
for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")


Label: 0.0, Count: 84406352
Label: 1.0, Count: 417566
Label: 2.0, Count: 633945
Label: 3.0, Count: 233239
Label: 4.0, Count: 11966
Label: 5.0, Count: 1188116
Label: 6.0, Count: 41613
Label: 7.0, Count: 79507
Label: 8.0, Count: 6494
Label: 9.0, Count: 105349
Label: 10.0, Count: 309209
Label: 11.0, Count: 8278802
Label: 12.0, Count: 122245
Label: 13.0, Count: 225692
Label: 14.0, Count: 16419
Label: 15.0, Count: 382875
Label: 16.0, Count: 9603


In [ ]:
import os
import glob
import nibabel as nib
import numpy as np

# Specify the directory containing NIfTI files
input_dir = '/content/drive/MyDrive/Neuro/CODE/seg'

# Get a list of NIfTI files in the directory
nifti_files = glob.glob(os.path.join(input_dir, '*.nii.gz'))

# Check if there are any files to process
if not nifti_files:
    print("No NIfTI files found in the specified directory.")
    exit()

# Dictionary to hold labels from each file
file_labels = {}

# Process each file
for file_path in nifti_files:
    # Load the NIfTI file
    nifti_img = nib.load(file_path)
    data_array = nifti_img.get_fdata()

    # Find unique labels in the segmentation
    unique_labels = np.unique(data_array)

    # Store the labels for this file
    file_labels[os.path.basename(file_path)] = set(unique_labels)

# Determine the set of common labels (intersection of all sets)
if file_labels:
    common_labels = set.intersection(*file_labels.values())
else:
    print("No labels found across files.")
    exit()

# Identify and report discrepancies
discrepancies = {}
for filename, labels in file_labels.items():
    if labels != common_labels:
        discrepancies[filename] = {
            'missing_labels': list(common_labels - labels),
            'extra_labels': list(labels - common_labels)
        }

# Output the discrepancies
if discrepancies:
    print("Discrepancies found in labels among files:")
    for filename, info in discrepancies.items():
        print(f"{filename}: Missing Labels: {info['missing_labels']}, Extra Labels: {info['extra_labels']}")
else:
    print("No discrepancies found. All files have the same set of labels.")


Discrepancies found in labels among files:
1927770.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
2009784.nii.gz: Missing Labels: [], Extra Labels: [8.0, 13.0, 6.0, 14.0]
2010014.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
2010213.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
2015772.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
1722078.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
1627009.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
1469159.nii.gz: Missing Labels: [], Extra Labels: [16.0, 13.0, 6.0, 14.0]
1640134.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
1601959.nii.gz: Missing Labels: [], Extra Labels: [6.0, 8.0, 13.0, 14.0, 16.0]
556631.nii.gz: Missing Labels: [], Extra Labels: [8.0, 16.0, 13.0, 14.0]
873308.nii.gz: Missing Labels: [], Extra Labels: [8.0, 13.0, 6.0, 14.0]
180818.nii.gz: Missing Labels: []

In [ ]:
print(common_labels)

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 9.0, 10.0, 11.0, 12.0, 15.0}


In [ ]:
# Get a list of .nrrd files in a directory
nrrd_files = glob.glob('/content/drive/MyDrive/Neuro/CODE/seg/*.nrrd')

# Loop through each .nrrd file
for nrrd_file in nrrd_files:
    # Read the .nrrd file
    data, header = nrrd.read(nrrd_file)

    # Create a NIfTI1Image object
    nifti_img = nib.Nifti1Image(data, affine=None)

    # Update the NIfTI header with necessary information
    nifti_img.header.set_data_dtype(data.dtype)
    nifti_img.header.set_zooms([header['space directions'][0, 0],
                            header['space directions'][1, 1],
                            header['space directions'][2, 2]])

    # Generate the output .nii file path by replacing the extension
    nii_file = nrrd_file.replace('.nrrd', '.nii.gz')

    # Save the NIfTI1Image object as .nii file
    nib.save(nifti_img, nii_file)

In [ ]:
import nrrd
import os

# Path to the input directory
input_dir = '/content/drive/MyDrive/Neuro/CODE/seg'

# List all nrrd files in the input directory
nrrd_files = [file for file in os.listdir(input_dir) if file.endswith('.nrrd')]

# Load the first nrrd file (for testing)
file_path = os.path.join(input_dir, nrrd_files[0])
data, header = nrrd.read(file_path)

# Print out the header to explore metadata
print(header)


OrderedDict([('type', 'unsigned char'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 200])), ('space directions', array([[0.58586719, 0.        , 0.        ],
       [0.        , 0.58586719, 0.        ],
       [0.        , 0.        , 0.8       ]])), ('kinds', ['domain', 'domain', 'domain']), ('encoding', 'gzip'), ('space origin', array([  -75.42906641,  -327.22306641, -1142.742     ])), ('Segment0_Color', '0.862745 0.470588 0.352941'), ('Segment0_ColorAutoGenerated', '1'), ('Segment0_Extent', '0 511 0 511 0 199'), ('Segment0_ID', 'Brain_Parenchyma'), ('Segment0_LabelValue', '1'), ('Segment0_Layer', '0'), ('Segment0_Name', 'brain parenchyma'), ('Segment0_NameAutoGenerated', '1'), ('Segment0_Tags', 'TerminologyEntry:Segmentation category and type - MONAI Auto3DSeg~SCT^123037004^Anatomical Structure~SCT^836432005^Parenchyma of brain~^^~Anatomic codes - DICOM master list~^^~^^|'), ('Segment10_Color', '0.145098 0.145098 0.0313725'), ('Segment10_ColorA

FUNCTION FOR CONVERTING A NRRD FILE INTO A .NII.GZ
If you want the uncompressed version (.NII) just change "output_file_name" ending in line 16 with ".nii" (instead of .nii.gz)

In [ ]:
import numpy as np
import nibabel as nib

# Output directory for the NIfTI files
output_dir = '/content/drive/MyDrive/Neuro/CODE/out'

# Labels we are interested in (example: 'brain parenchyma', 'thalamus')
interested_labels = ['lobar']

def save_label_as_nifti(data, header, label_value, output_dir, original_file_name, label_name):
    # Create a binary mask for the current label
    label_mask = data == label_value
    # Check if the mask has any true values
    if np.any(label_mask):
        # Construct the output file name
        output_file_name = f"{original_file_name}_{label_name.replace(' ', '_')}.nii.gz"
        print(f"Saving {output_file_name} with value {label_value}")
        # Convert binary mask to NIfTI image
        nifti_img = nib.Nifti1Image(label_mask.astype(np.int16), affine=np.eye(4))
        # Save the NIfTI image
        nib.save(nifti_img, os.path.join(output_dir, output_file_name))
    else:
        print(f"No data found for label {label_name} with value {label_value}")

for file in nrrd_files:
    print(f"Processing file: {file}")
    file_path = os.path.join(input_dir, file)
    data, header = nrrd.read(file_path)
    found_labels = False
    # Extracting the base name without extension to use in naming the output files
    base_name = os.path.splitext(file)[0]

    for i in range(100):  # Adjust based on the number of segments you expect
        label_name_key = f'Segment{i}_Name'
        label_value_key = f'Segment{i}_LabelValue'
        if label_name_key in header and header[label_name_key].lower() in interested_labels:
            label_name = header[label_name_key]
            label_value = int(header[label_value_key])
            save_label_as_nifti(data, header, label_value, output_dir, base_name, label_name)
            found_labels = True

    if not found_labels:
        print(f"No interested labels found in {file}")

Processing file: 1957445.nrrd
Saving 1957445_Lobar.nii.gz with value 11
Processing file: 1978753.nrrd
Saving 1978753_Lobar.nii.gz with value 11


In [ ]:
def save_label_as_nifti(data, header, label_value, output_dir, original_file_name, label_name):
    # Create a binary mask for the current label
    label_mask = data == label_value
    # Check if the mask has any true values
    if np.any(label_mask):
        # Print out some stats about the mask
        print(f"Label {label_name} with value {label_value} has {np.sum(label_mask)} true values.")
        # Construct the output file name with .nii.gz extension for compression
        output_file_name = f"{original_file_name}_{label_name.replace(' ', '_')}.nii.gz"
        print(f"Saving {output_file_name} with value {label_value}")
        # Convert binary mask to NIfTI image
        nifti_img = nib.Nifti1Image(label_mask.astype(np.int16), affine=np.eye(4))
        # Save the NIfTI image in compressed format
        nib.save(nifti_img, os.path.join(output_dir, output_file_name))
    else:
        print(f"No data found for label {label_name} with value {label_value}")

for file in nrrd_files:
    print(f"Processing file: {file}")
    file_path = os.path.join(input_dir, file)
    data, header = nrrd.read(file_path)
    found_labels = False
    # Extracting the base name without extension to use in naming the output files
    base_name = os.path.splitext(file)[0]

    for i in range(100):  # Adjust based on the number of segments you expect
        label_name_key = f'Segment{i}_Name'
        label_value_key = f'Segment{i}_LabelValue'
        if label_name_key in header and header[label_name_key].lower() in interested_labels:
            label_name = header[label_name_key]
            label_value = int(header[label_value_key])
            save_label_as_nifti(data, header, label_value, output_dir, base_name, label_name)
            found_labels = True

    if not found_labels:
        print(f"No interested labels found in {file}")

Processing file: 1957445.nrrd
Label Lobar with value 11 has 3107814 true values.
Saving 1957445_Lobar.nii.gz with value 11
Processing file: 1978753.nrrd
Label Lobar with value 11 has 4021631 true values.
Saving 1978753_Lobar.nii.gz with value 11
